In [1]:
import os
from pathlib import Path
import sys

if "__project_dir__" not in globals():
    __project_dir__ = Path.cwd().parents[1].resolve()

sys.path.append(__project_dir__)
os.chdir(__project_dir__)

In [5]:
import pandas as pd
import tidytcells as tt

In [3]:
df = pd.read_excel("/home/yutanagano/UCLOneDrive/MBPhD/projects/tcr_embedder/data/raw/kane/tcrs_cd4_cd8.xlsx")
df = df.rename(
    columns={
        "TRB_v_gene": "TRBV",
        "TRB_cdr3": "CDR3B",
        "TRB_j_gene": "TRBJ",
        "majorType": "type"
    }
)

In [4]:
df = df[["TRBV", "CDR3B", "TRBJ", "type"]]
df = df[df["type"].map(lambda x: x in ("CD4", "CD8"))]

In [6]:
df["TRBV"] = df["TRBV"].map(lambda x: tt.tcr.standardise(x, enforce_functional=True))
df["CDR3B"] = df["CDR3B"].map(tt.junction.standardise)
df["TRBJ"] = df["TRBJ"].map(lambda x: tt.tcr.standardise(x, enforce_functional=True))

/home/yutanagano/Projects/tcr_embedder/.venv/lib64/python3.8/site-packages/tidytcells/_utils/warnings.py:10: UserWarning: Failed to standardise: "TRBV5-7" for species homosapiens. Attempted fix "TRBV5-7" did not meet the standardised format requirements. Ignoring this gene name...
  warn(
/home/yutanagano/Projects/tcr_embedder/.venv/lib64/python3.8/site-packages/tidytcells/_utils/warnings.py:10: UserWarning: Failed to standardise: "TRBV6-7" for species homosapiens. Attempted fix "TRBV6-7" did not meet the standardised format requirements. Ignoring this gene name...
  warn(
/home/yutanagano/Projects/tcr_embedder/.venv/lib64/python3.8/site-packages/tidytcells/_utils/warnings.py:10: UserWarning: Failed to standardise: "TRBV23-1" for species homosapiens. Attempted fix "TRBV23-1" did not meet the standardised format requirements. Ignoring this gene name...
  warn(
/home/yutanagano/Projects/tcr_embedder/.venv/lib64/python3.8/site-packages/tidytcells/_utils/warnings.py:10: UserWarning: Failed

In [7]:
cleaned = df[df["TRBV"].notna() & df["CDR3B"].notna() & df["TRBJ"].notna()]

In [9]:
subset = cleaned.groupby("type").sample(n=10_000, random_state=420).reset_index(drop=True)

In [10]:
test = subset.groupby("type").sample(n=1000, random_state=420)
train = subset[~subset.index.isin(test.index)]

In [11]:
train.to_csv("/home/yutanagano/UCLOneDrive/MBPhD/projects/tcr_embedder/data/preprocessed/cd4_cd8/train.csv", index=False)
test.to_csv("/home/yutanagano/UCLOneDrive/MBPhD/projects/tcr_embedder/data/preprocessed/cd4_cd8/test.csv", index=False)